In [ ]:
%sql

drop table lc_loan_data

In [ ]:
# File location and type
file_location = "/FileStore/tables/LoanStats_2018Q4.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [ ]:
df.count()

In [ ]:
df.printSchema()

In [ ]:
temp_table_name = "loanstats"

df.createOrReplaceTempView(temp_table_name)

In [ ]:
%sql

select * from loanstats

In [ ]:
%sql
select count(*) from loanstats

In [ ]:
df.describe().show()

term	- The number of payments on the loan. Values are in months and can be either 36 or 60 <br/>
homeOwnership - The home ownership status provided by the borrower during registration. Our values are: RENT, OWN, MORTGAGE, OTHER. <br/>
grade	- LC assigned loan grade <br/>
purpose	- A category provided by the borrower for the loan request. <br/>
intRate	- Interest Rate on the loan <br/>
addrState	- The state provided by the borrower in the loan application <br/>
loan_status	- Current status of the loan <br/>
application_type - Indicates whether the loan is an individual application or a joint application with two co-borrowers <br/>
loan_amnt	- The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value. <br/>
emp_length	- Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. <br/>
annual_inc	- The self-reported annual income provided by the borrower during registration. <br/>
dti	- A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income. <br/>
dti_joint -	A ratio calculated using the co-borrowers' total monthly payments on the total debt obligations, excluding mortgages and the requested LC loan, divided by the co-borrowers' combined self-reported monthly income <br/>
delinq_2yrs	- The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years <br/>
revol_util	- Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit. <br/>
total_acc	- The total number of credit lines currently in the borrower's credit file <br/>
num_tl_90g_dpd_24m - 	Number of accounts 90 or more days past due in last 24 months<br/>

In [ ]:
df_sel = df.select("term", "home_ownership", "grade", "purpose", "int_rate", "installment","addr_state","loan_status","application_type","loan_amnt","emp_length", "annual_inc","dti","delinq_2yrs","revol_bal","revol_util","total_acc","num_tl_90g_dpd_24m","dti_joint")

In [ ]:
df_sel.describe().show()

In [ ]:
df_sel.describe("term","loan_amnt","emp_length", "annual_inc","dti","delinq_2yrs","revol_util","total_acc").show()

In [ ]:
df_sel.cache()

In [ ]:
df_sel.describe("loan_amnt","emp_length" ,"dti","delinq_2yrs","revol_util","total_acc").show()

In [ ]:
%sql
select distinct emp_length from loanstats limit 50

In [ ]:
from pyspark.sql.functions import regexp_replace, regexp_extract
from pyspark.sql.functions import col

regex_string='years|year|\\+|\\<'
df_sel.select(regexp_replace(col("emp_length"), regex_string, "").alias("emplength_cleaned"),col("emp_length")).show(10)

In [ ]:
regex_string="\\d+"
df_sel.select(regexp_extract(col("emp_length"), regex_string, 0).alias("emplength_cleaned"),col("emp_length")).show(10)

In [ ]:
df_sel.show(2)

In [ ]:
df_sel=df_sel.withColumn("term_cleaned",regexp_replace(col("term"), "months", "")).withColumn("emplen_cleaned",regexp_extract(col("emp_length"), "\\d+", 0))

In [ ]:
df_sel.select('term','term_cleaned','emp_length','emplen_cleaned').show(15)

In [ ]:
df_sel.printSchema()

In [ ]:
table_name="loanstatus_sel"

df_sel.createOrReplaceTempView(table_name)

In [ ]:
%sql
select * from loanstatus_sel

In [ ]:
df_sel.stat.cov('annual_inc', 'loan_amnt')

In [ ]:
df_sel.stat.corr('annual_inc', 'loan_amnt')

In [ ]:
%sql
select corr(loan_amnt, term_cleaned) as abc from loanstatus_sel

In [ ]:
df_sel.stat.crosstab('loan_status','grade').show()

In [ ]:
freq=df_sel.stat.freqItems(['purpose','grade'],0.3)


In [ ]:
freq.collect()

In [ ]:
df_sel.groupby('purpose').count().show()

In [ ]:
df_sel.groupby('purpose').count().orderBy(col('count').desc()).show()

In [ ]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max, avg

In [ ]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.05
df_sel.stat.approxQuantile("loan_amnt", quantileProbs, relError)

In [ ]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.0
df_sel.stat.approxQuantile("loan_amnt", quantileProbs, relError)

In [ ]:
quantileProbs = [0.25, 0.5, 0.75, 0.9]
relError = 0.5
df_sel.stat.approxQuantile("loan_amnt", quantileProbs, relError)

In [ ]:
from pyspark.sql.functions import isnan, when, count, col
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

In [ ]:
%sql

select loan_status, count(*) from loanstats group by loan_status order by 2 desc

In [ ]:
df_sel=df_sel.na.drop("all", subset=["loan_status"])

In [ ]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

In [ ]:
df_sel.count()

In [ ]:
df_sel.describe("dti","revol_util").show()

In [ ]:
%sql
select ceil(REGEXP_REPLACE(revol_util,"\%","")), count(*) from loanstatus_sel group by ceil(REGEXP_REPLACE(revol_util,"\%",""))

In [ ]:
%sql 

select * from loanstatus_sel where revol_util is null

In [ ]:
df_sel=df_sel.withColumn("revolutil_cleaned",regexp_extract(col("revol_util"), "\\d+", 0))

In [ ]:
df_sel.describe('revol_util','revolutil_cleaned').show()

In [ ]:
def fill_avg(df, colname): 
    return df.select(colname).agg(avg(colname))

In [ ]:
rev_avg=fill_avg(df_sel,'revolutil_cleaned')

In [ ]:
from pyspark.sql.functions import lit

rev_avg=fill_avg(df_sel,'revolutil_cleaned').first()[0]
df_sel=df_sel.withColumn('rev_avg',lit(rev_avg))

In [ ]:
from pyspark.sql.functions import coalesce
df_sel=df_sel.withColumn('revolutil_cleaned',coalesce(col('revolutil_cleaned'),col('rev_avg')))

In [ ]:
df_sel.describe('revol_util','revolutil_cleaned').show()

In [ ]:
df_sel=df_sel.withColumn("revolutil_cleaned",df_sel["revolutil_cleaned"].cast("double"))

In [ ]:
df_sel.describe('revol_util','revolutil_cleaned').show()

In [ ]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

In [ ]:
%sql 

select * from loanstatus_sel where dti is null

In [ ]:
%sql
select application_type, dti, dti_joint from loanstats where dti is null

In [ ]:
df_sel=df_sel.withColumn("dti_cleaned",coalesce(col("dti"),col("dti_joint")))

In [ ]:
df_sel.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_sel.columns]).show()

In [ ]:
df_sel.groupby('loan_status').count().show()

In [ ]:
df_sel.where(df_sel.loan_status.isin(["Late (31-120 days)", "Charged Off", "In Grace Period","Late (16-30 days)"])).show()

In [ ]:
df_sel=df_sel.withColumn("bad_loan", when(df_sel.loan_status.isin(["Late (31-120 days)", "Charged Off", "In Grace Period","Late (16-30 days)"]),'Yes').otherwise('No'))

In [ ]:
df_sel.groupBy('bad_loan').count().show()

In [ ]:
df_sel.filter(df_sel.bad_loan == 'Yes').show()

In [ ]:
df_sel.printSchema()

In [ ]:
df_sel_final=df_sel.drop('revol_util','dti','dti_joint')

In [ ]:
df_sel_final.printSchema()

In [ ]:
df_sel.stat.crosstab('bad_loan','grade').show()

In [ ]:
df_sel.describe('dti_cleaned').show()

In [ ]:
df_sel.filter(df_sel.dti_cleaned > 100).show()

In [ ]:
permanent_table_name = "lc_loan_data"

df_sel.write.format("parquet").saveAsTable(permanent_table_name)

In [ ]:
%sql
select * from lc_loan_data